First time setup code. This creates a folder in your GDrive and sets up folder structures. Once this is done you dont have to do it again.

In [7]:
from google.colab import drive
drive.mount("/content/drive")

import os, json
from pathlib import Path

# One place to store EVERYTHING for this project
PROJECT_ROOT = Path("/content/drive/MyDrive/CUT3R_project")

CFG_PATH = PROJECT_ROOT / "config.json"

# Default folder layout
cfg = {
    "project_root": str(PROJECT_ROOT),
    "inputs_dir": str(PROJECT_ROOT / "inputs"),   # videos go here
    "runs_dir": str(PROJECT_ROOT / "runs"),       # each run gets its own folder
}

# Create folders
Path(cfg["inputs_dir"]).mkdir(parents=True, exist_ok=True)
Path(cfg["runs_dir"]).mkdir(parents=True, exist_ok=True)

# Save config (so future cells always refer to the same place)
CFG_PATH.write_text(json.dumps(cfg, indent=2))
print("✅ Project initialized at:", cfg["project_root"])
print("📁 Put videos here:", cfg["inputs_dir"])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Project initialized at: /content/drive/MyDrive/CUT3R_project
📁 Put videos here: /content/drive/MyDrive/CUT3R_project/inputs


Every time you want to process a new video, you run this cell and only change VIDEO_NAME

In [8]:
import json, re, time
from pathlib import Path

cfg = json.loads(Path("/content/drive/MyDrive/CUT3R_project/config.json").read_text())

INPUTS_DIR = Path(cfg["inputs_dir"])
RUNS_DIR   = Path(cfg["runs_dir"])

# ✅ CHANGE THIS ONLY
VIDEO_NAME = "demovid.mov"   # put the exact filename in inputs/

VIDEO_PATH = INPUTS_DIR / VIDEO_NAME
assert VIDEO_PATH.exists(), f"Video not found: {VIDEO_PATH}"

def slugify(name: str) -> str:
    name = Path(name).stem
    name = re.sub(r"[^a-zA-Z0-9]+", "_", name).strip("_").lower()
    return name

# Run id = video slug + timestamp (prevents overwriting)
RUN_ID = f"{slugify(VIDEO_NAME)}_{time.strftime('%Y%m%d_%H%M%S')}"
RUN_DIR = RUNS_DIR / RUN_ID

FRAMES_DIR = RUN_DIR / "frames_2fps"
OUT_DIR    = RUN_DIR / "cut3r_out"

FRAMES_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("🎬 Video:", VIDEO_PATH)
print("🧪 Run ID:", RUN_ID)
print("🖼️ Frames:", FRAMES_DIR)
print("📦 Output:", OUT_DIR)

🎬 Video: /content/drive/MyDrive/CUT3R_project/inputs/demovid.mov
🧪 Run ID: demovid_20260114_215643
🖼️ Frames: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_215643/frames_2fps
📦 Output: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_215643/cut3r_out


Clone CUT3R

In [9]:
from pathlib import Path

REPO_DIR = Path("/content/CUT3R")
REPO_URL = "https://github.com/CUT3R/CUT3R.git"

%cd /content

# Clone once; reuse on reruns
if not REPO_DIR.exists():
    !git clone {REPO_URL} CUT3R

%cd /content/CUT3R

# Show what version you're on
!git rev-parse --short HEAD

/content
/content/CUT3R
8bc15dc


System deps + Python deps (stable / rerunnable)

In [10]:
%cd /content/CUT3R

# System dependency for video -> frames
!apt-get -qq update
!apt-get -qq install -y ffmpeg

# Python tooling you already used in your notebook
!python -m pip install -U pip
!pip -q install ninja gdown

# Install repo dependencies
!pip install -r requirements.txt

/content/CUT3R
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [11]:
# GPU visibility check
!nvidia-smi

import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
    print("cuda runtime:", torch.version.cuda)

Wed Jan 14 21:56:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Hardware + CUDA/Torch sanity checks

In [12]:
# GPU visibility check
!nvidia-smi

import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
    print("cuda runtime:", torch.version.cuda)


Wed Jan 14 21:57:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Check if we need the CUDA extension patch (fast check) [Note: If it prints nothing your good else do the patch+ rebuild CUDA extension command]

In [13]:
%cd /content/CUT3R
!grep -RIn "tokens\.type()" src/croco/models/curope || true

/content/CUT3R


patch + rebuild the CUDA extension (one time)

In [14]:
%cd /content/CUT3R

# Patch the deprecated call in the CUDA kernel
!sed -i 's/tokens\.type()/tokens\.scalar_type()/g' src/croco/models/curope/kernels.cu

# Rebuild the extension cleanly
!rm -rf src/croco/models/curope/build
!cd src/croco/models/curope && python setup.py build_ext --inplace

/content/CUT3R
running build_ext
W0114 21:57:10.279000 11261 torch/utils/cpp_extension.py:521] The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.6). Most likely this shouldn't be a problem.
W0114 21:57:10.280000 11261 torch/utils/cpp_extension.py:531] There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.5
building 'curope' extension
creating /content/CUT3R/src/croco/models/curope/build/temp.linux-x86_64-cpython-312
[1/2] c++ -MMD -MF /content/CUT3R/src/croco/models/curope/build/temp.linux-x86_64-cpython-312/curope.o.d -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.12/dist-packages/torch/include -I/usr/local/lib/python3.12/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/cuda/include -I/usr/include/python3.12 -c -c /content/CUT3R/src/croco/models/curope/curope.cpp -o 

below is a sanity check if you see a .so file then your good!

In [15]:
!ls -lah /content/CUT3R/src/croco/models/curope | head -n 50

total 11M
drwxr-xr-x 4 root root 4.0K Jan 14 22:01 .
drwxr-xr-x 4 root root 4.0K Jan 14 21:40 ..
drwxr-xr-x 4 root root 4.0K Jan 14 22:01 build
-rw-r--r-- 1 root root 1.3K Jan 14 21:25 curope2d.py
-rw-r--r-- 1 root root 2.5K Jan 14 21:25 curope.cpp
-rwxr-xr-x 1 root root  11M Jan 14 22:01 curope.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root  161 Jan 14 21:25 __init__.py
-rw-r--r-- 1 root root 3.9K Jan 14 21:57 kernels.cu
drwxr-xr-x 2 root root 4.0K Jan 14 21:40 __pycache__
-rw-r--r-- 1 root root 1.1K Jan 14 21:25 setup.py


Download weights

In [16]:
from pathlib import Path

WEIGHTS = Path("/content/CUT3R/src/cut3r_512_dpt_4_64.pth")

if WEIGHTS.exists():
    print("✅ Weights already present:", WEIGHTS)
else:
    %cd /content/CUT3R/src
    !gdown --fuzzy "https://drive.google.com/file/d/1Asz-ZB3FfpzZYwunhQvNPZEUA8XUNAYD/view"
    print("✅ Download complete")

%cd /content/CUT3R
!ls -lh /content/CUT3R/src/*.pth

✅ Weights already present: /content/CUT3R/src/cut3r_512_dpt_4_64.pth
/content/CUT3R
-rw-r--r-- 1 root root 3.0G Feb  7  2025 /content/CUT3R/src/cut3r_512_dpt_4_64.pth


Shows the project structure in Drive

In [17]:
from pathlib import Path
import json

PROJECT_ROOT = Path("/content/drive/MyDrive/CUT3R_project")
CFG_PATH = PROJECT_ROOT / "config.json"

cfg = {
    "project_root": str(PROJECT_ROOT),
    "inputs_dir": str(PROJECT_ROOT / "inputs"),
    "runs_dir": str(PROJECT_ROOT / "runs"),
}

Path(cfg["inputs_dir"]).mkdir(parents=True, exist_ok=True)
Path(cfg["runs_dir"]).mkdir(parents=True, exist_ok=True)

CFG_PATH.write_text(json.dumps(cfg, indent=2))
print("✅ Using project root:", cfg["project_root"])
print("📁 Put videos here:", cfg["inputs_dir"])


✅ Using project root: /content/drive/MyDrive/CUT3R_project
📁 Put videos here: /content/drive/MyDrive/CUT3R_project/inputs


Per-run video selection make sure to edit videoname

In [18]:
import json, re, time
from pathlib import Path

cfg = json.loads(Path("/content/drive/MyDrive/CUT3R_project/config.json").read_text())

INPUTS_DIR = Path(cfg["inputs_dir"])
RUNS_DIR   = Path(cfg["runs_dir"])

# ✅ CHANGE THIS ONLY (must match the exact filename in inputs/)
VIDEO_NAME = "demovid.mov"

VIDEO_PATH = INPUTS_DIR / VIDEO_NAME
assert VIDEO_PATH.exists(), f"Video not found: {VIDEO_PATH}"

def slugify(name: str) -> str:
    name = Path(name).stem
    name = re.sub(r"[^a-zA-Z0-9]+", "_", name).strip("_").lower()
    return name

RUN_ID = f"{slugify(VIDEO_NAME)}_{time.strftime('%Y%m%d_%H%M%S')}"
RUN_DIR = RUNS_DIR / RUN_ID

FRAMES_DIR = RUN_DIR / "frames_2fps"
OUT_DIR    = RUN_DIR / "cut3r_out"

FRAMES_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("🎬 Video:", VIDEO_PATH)
print("🧪 Run ID:", RUN_ID)
print("🖼️ Frames:", FRAMES_DIR)
print("📦 Output:", OUT_DIR)

🎬 Video: /content/drive/MyDrive/CUT3R_project/inputs/demovid.mov
🧪 Run ID: demovid_20260114_220106
🖼️ Frames: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/frames_2fps
📦 Output: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out


check to make sure video is in right folder

In [19]:
!ls -lah "/content/drive/MyDrive/CUT3R_project/inputs"


total 1.7M
-rw------- 1 root root 1.7M Jan 14 21:18 demovid.mov


extract frames into FRAMES_DIR

In [20]:
# Clear old frames for this run (just in case)
!rm -f "{FRAMES_DIR}"/*.jpg

# Extract at 2 fps and scale width to 960 (keeps memory reasonable)
!ffmpeg -i "{VIDEO_PATH}" -vf "fps=2,scale=960:-1" "{FRAMES_DIR}/%06d.jpg"

!ls -lah "{FRAMES_DIR}" | head -n 20
!ls "{FRAMES_DIR}" | wc -l

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Patches that loader so the checkpoint can actually be read

In [21]:
from pathlib import Path
import re

path = Path("/content/CUT3R/src/dust3r/model.py")
txt = path.read_text()

# 0) Backup once
bak = path.with_suffix(".py.bak")
if not bak.exists():
    bak.write_text(txt)

# 1) Add allowlist for OmegaConf DictConfig (only if not already added)
if "add_safe_globals" not in txt:
    # Insert right after 'import torch'
    txt = re.sub(
        r"(import torch\s*\n)",
        r"\1from torch.serialization import add_safe_globals\n"
        r"import omegaconf\n"
        r"add_safe_globals([omegaconf.dictconfig.DictConfig])\n",
        txt,
        count=1
    )

# 2) Force torch.load(..., weights_only=False) for this repo's load_model path
# Replace the specific call that is failing: torch.load(model_path, map_location="cpu")
txt = txt.replace(
    'torch.load(model_path, map_location="cpu")',
    'torch.load(model_path, map_location="cpu", weights_only=False)'
)

path.write_text(txt)
print("✅ Patched:", path)


✅ Patched: /content/CUT3R/src/dust3r/model.py


Run Cut3R on the frame sequence

Details:
--model_path ...pth
Loads the pretrained Cut3R weights you downloaded (3.0GB file).

--size 512
Sets the internal processing resolution (higher = more detail, more VRAM/time).
With a T4 and 14 frames, 512 should be fine.

--seq_path "{FRAMES_DIR}"
Points Cut3R at the folder of numbered JPG frames (000001.jpg, etc).

--vis_threshold 1.5
A visualization / filtering threshold used by the demo to decide what to show/keep.

--output_dir "{OUT_DIR}"
Where all outputs (predictions, visualizations, any 3D artifacts) get written.

IF IT KEEPS RUNNING WITH A LOCAL HOST LINK STOP THE SCRIPT THE MODEL HAS BEEN SAVED AND YOU ARE GOOD TO GO! :)

In [22]:
%cd /content/CUT3R

!python demo.py \
  --model_path /content/CUT3R/src/cut3r_512_dpt_4_64.pth \
  --size 512 \
  --seq_path "{FRAMES_DIR}" \
  --vis_threshold 1.5 \
  --output_dir "{OUT_DIR}"

/content/CUT3R
2026-01-14 22:01:14.489828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768428074.514590   12405 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768428074.524622   12405 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768428074.551079   12405 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768428074.551110   12405 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768428074.551116   12405 computation_placer.cc:177] computa

Confirm we got the right output

In [23]:
!ls -lah "{OUT_DIR}"
!find "{OUT_DIR}" -maxdepth 3 -type f | head -n 80

total 16K
drwx------ 2 root root 4.0K Jan 14 22:01 camera
drwx------ 2 root root 4.0K Jan 14 22:01 color
drwx------ 2 root root 4.0K Jan 14 22:01 conf
drwx------ 2 root root 4.0K Jan 14 22:01 depth
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000000.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000001.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000002.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000003.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000004.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000005.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000006.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/depth/000007.npy
/content/drive/MyDrive/CUT3R_project/runs/demovid_20260114

In [24]:
import os, glob
from pathlib import Path

out = Path(str(OUT_DIR))  # uses your OUT_DIR variable from earlier
print("OUT_DIR:", out)

for sub in ["camera","color","conf","depth"]:
    files = sorted((out/sub).glob("*"))
    print(f"\n{sub}/  ({len(files)} files)")
    print("  examples:", [f.name for f in files[:5]])


OUT_DIR: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out

camera/  (14 files)
  examples: ['000000.npz', '000001.npz', '000002.npz', '000003.npz', '000004.npz']

color/  (14 files)
  examples: ['000000.png', '000001.png', '000002.png', '000003.png', '000004.png']

conf/  (14 files)
  examples: ['000000.npy', '000001.npy', '000002.npy', '000003.npy', '000004.npy']

depth/  (14 files)
  examples: ['000000.npy', '000001.npy', '000002.npy', '000003.npy', '000004.npy']


Inspect one camera file to know the keys

In [25]:
import numpy as np
from pathlib import Path

out = Path(str(OUT_DIR))
sample = out / "camera" / "000000.npz"
cam = np.load(sample)

print("camera keys:", cam.files)
for k in cam.files:
    v = cam[k]
    print(k, "shape:", v.shape, "dtype:", v.dtype)


camera keys: ['pose', 'intrinsics']
pose shape: (4, 4) dtype: float32
intrinsics shape: (3, 3) dtype: float32


Install Open3D

In [26]:
!pip -q uninstall -y numpy
!pip -q install "numpy<2.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.


In [27]:
!pip -q install open3d==0.19.0

In [28]:
# Check to see if open3d is installed properly
import open3d as o3d
print("open3d:", o3d.__version__)

open3d: 0.19.0


Fuse into point cloud + export PLY

In [29]:
import numpy as np
import open3d as o3d
import imageio.v2 as imageio
from pathlib import Path

out = Path(str(OUT_DIR))

color_files = sorted((out/"color").glob("*.png"))
depth_files = sorted((out/"depth").glob("*.npy"))
conf_files  = sorted((out/"conf").glob("*.npy"))
cam_files   = sorted((out/"camera").glob("*.npz"))

assert len(color_files) == len(depth_files) == len(conf_files) == len(cam_files), "Mismatched counts"
print("Frames:", len(color_files))

# ---- tuning knobs ----
MAX_DEPTH_M = 10.0     # ignore farther points
CONF_THRESH = 0.30     # raise for cleaner, lower for denser
STRIDE = 1             # set 2 to use every other frame if you want
VOXEL = 0.01           # meters; increase for more smoothing

pcd_all = o3d.geometry.PointCloud()

for i in range(0, len(color_files), STRIDE):
    color = imageio.imread(color_files[i])[:, :, :3].astype(np.float32) / 255.0
    depth = np.load(depth_files[i]).astype(np.float32)
    conf  = np.load(conf_files[i]).astype(np.float32)

    cam = np.load(cam_files[i])
    K = cam["intrinsics"].astype(np.float32)
    T = cam["pose"].astype(np.float32)

    # mask good pixels
    m = (depth > 0) & (depth < MAX_DEPTH_M) & (conf >= CONF_THRESH)
    ys, xs = np.where(m)
    z = depth[ys, xs]

    fx, fy = K[0,0], K[1,1]
    cx, cy = K[0,2], K[1,2]

    # backproject to camera coords
    x = (xs - cx) * z / fx
    y = (ys - cy) * z / fy
    pts_cam = np.stack([x, y, z], axis=1)

    cols = color[ys, xs]

    # camera -> world transform
    pts_h = np.concatenate([pts_cam, np.ones((pts_cam.shape[0], 1), dtype=np.float32)], axis=1)
    pts_w = (pts_h @ T.T)[:, :3]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pts_w)
    pcd.colors = o3d.utility.Vector3dVector(cols)

    pcd_all += pcd

raw_n = np.asarray(pcd_all.points).shape[0]
print("Raw points:", raw_n)

# downsample + denoise
pcd_all = pcd_all.voxel_down_sample(voxel_size=VOXEL)
pcd_all, _ = pcd_all.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

final_n = np.asarray(pcd_all.points).shape[0]
print("After cleanup:", final_n)

ply_path = out / "fused_pointcloud.ply"
o3d.io.write_point_cloud(str(ply_path), pcd_all)
print("✅ Wrote:", ply_path)

Frames: 14
Raw points: 1979465
After cleanup: 1594298
✅ Wrote: /content/drive/MyDrive/CUT3R_project/runs/demovid_20260114_220106/cut3r_out/fused_pointcloud.ply
